In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

dftrain = pd.read_csv('data/extras/train.csv', header=None)
dftest = pd.read_csv('data/extras/test.csv', header=None)

In [32]:
df_train = dftrain.iloc[:, 1:]

scaler = StandardScaler()
# features = ['AvgPointsPerGame', 'atHome', '7GameAvg', 'FloorAvg']

X_train = scaler.fit_transform(df_train.iloc[:,:-1].values)
y_train = df_train.iloc[:,-1].values
#y_train_onehot = pd.get_dummies(df_train['4GameAvg']).values

In [33]:
df_test = dftest.iloc[:, 1:]

X_test = scaler.transform(df_test.iloc[:, :-1].values)
y_test = df_test.iloc[:, -1].values

In [34]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=0, verbose=3)
model = model.fit(X_train, y_train)

y_prediction = model.predict(X_test)
print ("\naccuracy", np.sum(abs(y_prediction-y_test)<5) / float(len(y_test)))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.487640449438


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [45]:
np.sum((y_prediction/y_test)<10)/float(len(y_test))

ValueError: operands could not be broadcast together with shapes (445,2) (445,) 

In [72]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(12, input_dim=38, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
          
          
model.compile(optimizer='sgd', metrics=['accuracy'],loss="mean_squared_error")

model.fit(X_train, y_train, 
          validation_data=(X_test,y_test),
          verbose=False)

print ('\ntime taken %s seconds' % str(time() - start))


time taken 0.6524238586425781 seconds


In [73]:
scores = model.evaluate(X_train,y_train)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/445 [=>............................] - ETA: 0s

416/445 [===========================>..] - ETA: 0s

acc: 0.67%


In [74]:
y_prediction = model.predict(X_train)

In [75]:
y_prediction

array([[ 0.99999237],
       [ 0.9609983 ],
       [ 0.99983823],
       [ 0.98180032],
       [ 0.99079728],
       [ 0.98044753],
       [ 0.99097919],
       [ 0.99993837],
       [ 0.98770881],
       [ 0.96385992],
       [ 0.91424108],
       [ 0.98743457],
       [ 0.98046517],
       [ 0.9831987 ],
       [ 0.98906124],
       [ 0.99994731],
       [ 0.99999976],
       [ 0.96314794],
       [ 0.94168222],
       [ 0.98992389],
       [ 0.92021793],
       [ 0.98883444],
       [ 0.98528779],
       [ 0.99972373],
       [ 0.99144506],
       [ 0.99992788],
       [ 0.97083843],
       [ 0.98859465],
       [ 0.98608649],
       [ 0.99739909],
       [ 0.98612761],
       [ 0.99961805],
       [ 0.98538256],
       [ 0.99939716],
       [ 0.95139599],
       [ 0.9697206 ],
       [ 0.96667743],
       [ 0.98197007],
       [ 0.98722851],
       [ 0.97321653],
       [ 0.98010892],
       [ 0.99999845],
       [ 0.96874362],
       [ 0.95216531],
       [ 0.98904514],
       [ 0